In [ ]:
import pyodbc
import pymysql
import time
from datetime import datetime
import threading
import signal
import sys

# ================== تنظیمات ==================
INTERVAL_SECONDS = 60 * 60  # هر ۶۰ دقیقه
RUNNING = True

# ================== اتصالات ==================
def get_sql_connection():
    return pyodbc.connect(
        'DRIVER={SQL Server};'
        'SERVER=MKZ-DSAS\\DSAS;'
        'DATABASE=DSAS;'
        'UID=datadriven;'
        'PWD=5Rdx@4Rfv1355'
    )

def get_mysql_connection():
    return pymysql.connect(
        host="127.0.0.1",
        port=3306,
        user="root",
        password="",
        database="dsas",
        charset="utf8mb4",
        cursorclass=pymysql.cursors.DictCursor
    )

# ================== تنظیمات جدید ==================
TABLE_NAME = "main_table_mhi_bearings_temp_vibration_12"
UNIT_ID = 12

# AssetID اصلی که بر اساس آن رکورد جدید ساخته می‌شود
MAIN_ASSET_ID = 9358

# لیست تمام AssetIDهای فرعی که باید پر شوند
SUB_ASSET_IDS = list(range(9359, 9375))  # 9359 تا 9374

# نگاشت AssetID → نام فیلد در جدول MySQL
field_map = {aid: f"AssetID_{aid}" for aid in [MAIN_ASSET_ID] + SUB_ASSET_IDS}

# ================== تابع اصلی ==================
def run_task():
    print(f"\n{'='*80}")
    print(f"شروع اجرا: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"جدول هدف → {TABLE_NAME}")
    print(f"UnitID = {UNIT_ID} | AssetID اصلی = {MAIN_ASSET_ID}")
    print(f"AssetIDهای فرعی → {SUB_ASSET_IDS}")
    print(f"{'='*80}")

    conn_sql = None
    conn_mysql = None
    cursor_sql = None

    try:
        # اتصال‌ها
        print("اتصال به SQL Server...")
        conn_sql = get_sql_connection()
        cursor_sql = conn_sql.cursor()

        print("اتصال به MySQL...")
        conn_mysql = get_mysql_connection()

        # خواندن TimeStampsهای موجود در جدول MySQL
        print("در حال دریافت TimeStampsهای موجود در جدول MySQL...")
        existing_ts = set()
        with conn_mysql.cursor() as c:
            c.execute(f"SELECT TimeStamps FROM {TABLE_NAME} WHERE unitID = %s", (UNIT_ID,))
            for row in c.fetchall():
                existing_ts.add(row['TimeStamps'])
        print(f"تعداد رکوردهای موجود: {len(existing_ts):,} رکورد")

        # گرفتن آخرین ۱۲۰۰۰ رکورد از AssetID اصلی
        print(f"در حال دریافت آخرین ۱۲۰۰۰ رکورد برای AssetID={MAIN_ASSET_ID} ...")
        query_main = f"""
            SELECT TOP 12000 [Value], [RecordTime], [RecordDate], [DateTime], [TimeStamp]
            FROM [DSAS].[PDA].[Periodic_Values]
            WHERE [UnitID] = ? AND [AssetID] = ?
            ORDER BY [DateTime] DESC
        """
        cursor_sql.execute(query_main, (UNIT_ID, MAIN_ASSET_ID))
        rows_main = cursor_sql.fetchall()

        if not rows_main:
            print(f"هیچ داده‌ای برای AssetID={MAIN_ASSET_ID} پیدا نشد.")
            return

        inserted_count = 0
        for idx, row in enumerate(rows_main):
            ts = row.TimeStamp
            if ts in existing_ts:
                if idx % 100 == 0:  # فقط هر ۱۰۰ تا یکبار چاپ کنه که خروجی شلوغ نشه
                    print(f"رکورد {idx+1:,}: TimeStamps={ts} قبلاً وجود دارد → رد شد.")
                continue

            dt = row.DateTime
            main_value = float(row.Value) if row.Value is not None else None

            print(f"\n{'-'*50}")
            print(f"درج رکورد جدید {idx+1:,} | TimeStamps = {ts} | DateTime = {dt}")

            # ۱. درج رکورد اصلی (فقط AssetID_9358 پر می‌شود)
            insert_sql = f"""
                INSERT INTO {TABLE_NAME} 
                (unitID, AssetID_9358, RecordTime, RecordDate, DateTime, TimeStamps)
                VALUES (%s, %s, %s, %s, %s, %s)
            """
            try:
                with conn_mysql.cursor() as c:
                    c.execute(insert_sql, (
                        UNIT_ID,
                        main_value,
                        dt.strftime('%H:%M:%S'),
                        dt.strftime('%Y-%m-%d'),
                        dt,
                        ts
                    ))
                conn_mysql.commit()
                inserted_count += 1
                print("رکورد اصلی با موفقیت درج شد.")
            except Exception as e:
                print(f"خطا در درج رکورد اصلی: {e}")
                conn_mysql.rollback()
                continue

            # ۲. پر کردن فیلدهای فرعی (9359 تا 9374)
            for aid in SUB_ASSET_IDS:
                field_name = field_map[aid]

                # جستجوی نزدیک (اختلاف ≤ ۱۰۰۰)
                near_sql = f"""
                    SELECT TOP 1 [Value]
                    FROM [DSAS].[PDA].[Periodic_Values]
                    WHERE [UnitID]=? AND [AssetID]=? AND ABS([TimeStamp] - ?) <= 1000
                    ORDER BY ABS([TimeStamp] - ?)
                """
                cursor_sql.execute(near_sql, (UNIT_ID, aid, ts, ts))
                near_row = cursor_sql.fetchone()

                if near_row and near_row.Value is not None:
                    value = float(near_row.Value)
                    source = "نزدیک"
                else:
                    # میانگین ۱۰ مقدار آخر
                    avg_sql = f"""
                        SELECT AVG(CAST([Value] AS FLOAT))
                        FROM (
                            SELECT TOP 10 [Value]
                            FROM [DSAS].[PDA].[Periodic_Values]
                            WHERE [UnitID]=? AND [AssetID]=?
                            ORDER BY [DateTime] DESC
                        ) sub
                    """
                    cursor_sql.execute(avg_sql, (UNIT_ID, aid))
                    avg_row = cursor_sql.fetchone()
                    value = round(float(avg_row[0]), 4) if avg_row and avg_row[0] is not None else None
                    source = "میانگین ۱۰ تایی"

                if value is not None:
                    update_sql = f"UPDATE {TABLE_NAME} SET {field_name} = %s WHERE TimeStamps = %s"
                    try:
                        with conn_mysql.cursor() as c:
                            c.execute(update_sql, (value, ts))
                        conn_mysql.commit()
                        print(f"  {field_name} = {value}  ({source})")
                    except Exception as e:
                        print(f"  خطا در آپدیت {field_name}: {e}")
                        conn_mysql.rollback()
                else:
                    print(f"  {field_name}: داده‌ای یافت نشد.")

        print(f"\n{'='*80}")
        print(f"خلاصه این اجرا: {inserted_count:,} رکورد جدید درج و آپدیت شد.")
        print(f"{'='*80}")

    except Exception as e:
        print(f"خطای کلی: {e}")
    finally:
        if cursor_sql: cursor_sql.close()
        if conn_sql: conn_sql.close()
        if conn_mysql: conn_mysql.close()

# ================== توقف با Ctrl+C ==================
def signal_handler(sig, frame):
    global RUNNING
    print('\nدریافت سیگنال توقف (Ctrl+C). در حال خروج...')
    RUNNING = False
    sys.exit(0)

signal.signal(signal.SIGINT, signal_handler)

# ================== حلقه زمان‌بندی ==================
def scheduler():
    while RUNNING:
        run_task()
        print(f"\nخواب برای {INTERVAL_SECONDS // 60} دقیقه بعدی (تا {datetime.now() + timedelta(seconds=INTERVAL_SECONDS)})")
        for _ in range(INTERVAL_SECONDS):
            if not RUNNING:
                break
            time.sleep(1)

if __name__ == "__main__":
    scheduler()


شروع اجرا: 2025-11-20 08:30:52
جدول هدف → main_table_mhi_bearings_temp_vibration_12
UnitID = 12 | AssetID اصلی = 9358
AssetIDهای فرعی → [9359, 9360, 9361, 9362, 9363, 9364, 9365, 9366, 9367, 9368, 9369, 9370, 9371, 9372, 9373, 9374]
اتصال به SQL Server...
اتصال به MySQL...
در حال دریافت TimeStampsهای موجود در جدول MySQL...
تعداد رکوردهای موجود: 0 رکورد
در حال دریافت آخرین ۱۲۰۰۰ رکورد برای AssetID=9358 ...

--------------------------------------------------
درج رکورد جدید 1 | TimeStamps = 1763541726 | DateTime = 2025-11-19 12:12:06
رکورد اصلی با موفقیت درج شد.
  AssetID_9359 = 86.0  (نزدیک)
  AssetID_9360 = 58.0  (نزدیک)
  AssetID_9361 = 22.0  (نزدیک)
  AssetID_9362 = 67.0  (نزدیک)
  AssetID_9363 = 36.0  (نزدیک)
  AssetID_9364 = 88.0  (نزدیک)
  AssetID_9365 = 47.0  (نزدیک)
  AssetID_9366 = 28.0  (نزدیک)
  AssetID_9367 = 51.0  (نزدیک)
  AssetID_9368 = 75.0  (نزدیک)
  AssetID_9369 = 70.0  (نزدیک)
  AssetID_9370 = 72.0  (نزدیک)
  AssetID_9371 = 61.0  (نزدیک)
  AssetID_9372 = 60.0  (نزدیک)